In [1]:
import json
import jellyfish
import numpy
import pandas as pd
import math

In [2]:
#import snakemake rules
with open('/home/marinedjaffardjy/Documents/Code/Similarite_process/snk_rule_info.json') as f:
    snk_rules_all = json.load(f)
#import snakemake rule with tools
with open('json/snk_rule_info_tool.json') as f:
    snk_tools = json.load(f)

In [3]:
with open("/home/marinedjaffardjy/Documents/Code/Similarite_process/json/nf_all_proc.json") as f:
    nf_rules_all = json.load(f)
    
with open("/home/marinedjaffardjy/Documents/Code/Similarite_process/json/nf_proc_tool.json") as f:
    nf_rules_tools = json.load(f)
    

In [4]:
len(nf_rules_tools)

9652

In this notebook we'll compute the levenstein distance for :
- snakemake processes that contian at least one tool
- all snakemake processes
- all snakemake workflows
- nextflow processes with tools
- all nextflow processes with tools
- all nextflow workflows

In [4]:
#compute levenshtein for snakemake
def levenshtein_snk(snk_rules, resume=0, output_file = "/home/marinedjaffardjy/Documents/Code/Similarite_process/json/levenshtein/levenshtein_tool_snk", outputfile_resume = "/home/marinedjaffardjy/Documents/Code/Similarite_process/json/levenshtein_tools_snk_587.json"):
    #input : 
    #    list of snakemake rules in form of a json
    #    resume : int at which the computation will be restarted
    #    output_file : string model of the names for the output files
    #    output_file_resume : string file for the scores already computed in the case of a resume
    #output : table of dict with scores. also saves the scores for all pairs of processes in json (path )
    v1 = snk_rules.copy()[resume:-1]
    v2 = snk_rules.copy()[resume+1:]
    i = 0
    if(resume>0):
        with open(outputfile_resume) as f:
            scores = json.load(f)
            f.close
    else :
        scores = []
    for rule1 in v1:
        print(str(i)+"/"+str(len(v1)))
        i+=1
        for rule2 in v2:
            score = jellyfish.levenshtein_distance(rule1['code'],rule2['code'])
            l = max(len(rule1['code']),len(rule2['code']))
            scores.append({"rule1":rule1["wf_orig"]+"/"+rule1["name"],
                           "rule2":rule2["wf_orig"]+"/"+rule2["name"],
                           "levenshtein":(l-score)/l})
        if(len(v2)>=2):
            v2 = v2[1:]
        if(i%50==0 or i ==len(v1) ):
            with open(output_file+"_"+str(i+resume)+".json","w") as f:
                print(output_file+"_"+str(i+resume)+".json")
                json.dump(scores,f)
                f.close
    return scores

In [ ]:
%%time
#levenstein on snk with tools
scores = levenshtein_snk(snk_tools,0,"/home/marinedjaffardjy/Documents/Code/Similarite_process/json/levenshtein_tools_snk")

In [ ]:
%%time
#levenstein on snk all
with open('/home/marinedjaffardjy/Documents/Code/Investigating_reuse/json/snk_rule_info.json') as f:
    snk_tools = json.load(f)
scores = levenshtein_snk(snk_tools,resume=450,output_file = "/home/marinedjaffardjy/Documents/Code/Similarite_process/json/levenshtein_snk",outputfile_resume = "/home/marinedjaffardjy/Documents/Code/Similarite_process/json/levenshtein_snk_450.json")

0/26250
1/26250
2/26250
3/26250
4/26250
5/26250
6/26250
7/26250
8/26250
9/26250
10/26250
11/26250
12/26250
13/26250
14/26250
15/26250
16/26250
17/26250
18/26250
19/26250
20/26250
21/26250
22/26250
23/26250
24/26250
25/26250
26/26250
27/26250
28/26250
29/26250
30/26250
31/26250
32/26250
33/26250
34/26250
35/26250
36/26250
37/26250
38/26250
39/26250
40/26250
41/26250
42/26250
43/26250
44/26250
45/26250
46/26250
47/26250
48/26250
49/26250
/home/marinedjaffardjy/Documents/Code/Similarite_process/json/levenshtein_snk_500.json
50/26250
51/26250
52/26250
53/26250
54/26250
55/26250
56/26250
57/26250
58/26250
59/26250
60/26250
61/26250
62/26250
63/26250
64/26250
65/26250
66/26250
67/26250
68/26250
69/26250
70/26250
71/26250
72/26250
73/26250
74/26250
75/26250
76/26250
77/26250
78/26250
79/26250
80/26250
81/26250
82/26250
83/26250
84/26250
85/26250
86/26250
87/26250
88/26250
89/26250
90/26250
91/26250
92/26250
93/26250
94/26250
95/26250
96/26250
97/26250
98/26250
99/26250
/home/marinedjaffardjy/

Let us compute the levenshtein distance for all snakemake whole workflows

In [ ]:
%%time
#levenstein on snk all
with open('/home/marinedjaffardjy/Documents/Code/Investigating_reuse/json/snk_rule_info_tool.json') as f:
    all_wf = json.load(f)
scores = levenshtein_snk(snk_rules)

In [105]:
path = "/home/marinedjaffardjy/Bureau/simil_code/text_modif"
# Change the directory
os.chdir(path)
  
# Read text File
  
def read_text_file(file_path):
    with open(file_path, 'r') as f:
        return f.read()
  
  
# iterate through all file
files_path = []
for file in os.listdir():
    # Check whether file is in text format or not
    if file.endswith(".txt"):
        files_path.append(f"{path}/{file}")
  
        # call read text file function
files_path.sort()
files=[]
for file_path in files_path:
    files.append(read_text_file(file_path))

In [13]:
nf_rules_no_tools=[]
for el in nf_rules_all:
    if len(el['tools'])==0:
        nf_rules_no_tools.append(el)
with open("/home/marinedjaffardjy/Documents/Code/Similarite_process/json/nf_rules_no_tools.json",'w') as f:
    json.dump(nf_rules_no_tools,f)
    
snk_rules_no_tools=[]
for el in snk_rules_all:
    if len(el['tools'])==0:
        snk_rules_no_tools.append(el)
with open("/home/marinedjaffardjy/Documents/Code/Similarite_process/json/snk_rules_no_tools.json",'w') as f:
    json.dump(snk_rules_no_tools,f)